# 📍 NUT에 징발되신 여러분을 위한 예약 봇!!!

### 먼저 셀레니움 임포트 & 페이지 로드할게용

In [142]:
#셀레니움 임포트
from selenium import webdriver # 자바스크립트로 전송 되는 데이터는 url - html에 안붙어있습니다.. 페이지를 열어줘야 데이터가 와요!
from selenium.webdriver.chrome.options import Options 
import time
import requests
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup as bs
import lxml
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

#드라이버 열기
path = './chromedriver' #패스는 여러분 크롬드라이버가 있는 곳으로 설정해주세요~~ 저는 같은 폴더에 있어서

chrome_options = Options()
chrome_options.set_capability('unhandledPromptBehavior', 'accept')  #이거 없으면 계속 오류뜨더라고요...

driver = webdriver.Chrome(path,options=chrome_options)

#로그인하기 - 근데 로그인은 수동으로 하셔도 되지 않을까.. 싶습니다
driver.get("https://sso.snu.ac.kr/3rdParty/loginFormPage.jsp?NONCE=4bhjtiICB%2BgCd7SHcyHx5s5mA2dZh7sbfq3p8cC4q%2FDSmBqMTKVfRieMdwcaip3QF0YzX3vfTO5mStthdOOUew%3D%3D&UURL=https%3A%2F%2Fsso.snu.ac.kr%2Fnls3%2Ffcs")
userId = driver.find_element(By.ID, 'si_id')
userId.send_keys('여러분 아이디 여러분 아이디 여러분 아이디 ') # 로그인 할 계정 id
userPwd = driver.find_element(By.ID, 'si_pwd')
userPwd.send_keys('비번 비번 비번 비번') # 로그인 할 계정의 패스워드 
userPwd.send_keys(Keys.ENTER)

#스터디룸 예약 페이지 접속
driver.get("https://eng.snu.ac.kr/studyroom")

### 세션 날짜들 모음입니다

In [143]:
days = ['2021년 3월 2일',
 '2021년 3월 9일',
 '2021년 3월 16일',
 '2021년 3월 23일',
 '2021년 3월 30일',
 '2021년 3월 4일',
 '2021년 3월 11일',
 '2021년 3월 18일',
 '2021년 3월 25일',
 '2021년 4월 6일',
 '2021년 4월 13일',
 '2021년 4월 20일',
 '2021년 4월 27일',
 '2021년 4월 1일',
 '2021년 4월 8일',
 '2021년 4월 15일',
 '2021년 4월 22일',
 '2021년 4월 29일',
 '2021년 5월 4일',
 '2021년 5월 11일',
 '2021년 5월 18일',
 '2021년 5월 25일',
 '2021년 5월 6일',
 '2021년 5월 13일',
 '2021년 5월 20일',
 '2021년 5월 27일',
 '2021년 6월 1일',
 '2021년 6월 8일',
 '2021년 6월 15일',
 '2021년 6월 22일',
 '2021년 6월 29일',
 '2021년 6월 3일',
 '2021년 6월 10일',
 '2021년 6월 17일',
 '2021년 6월 24일']
sats = ['2021년 3월 6일',
 #'2021년 3월 13일',     #참고로 3월 13일에 다른 학회인지 예약해뒀더라구요! 중복되면 이상한데 예약하니까 일단 3월 13일은 빼뒀습니다.
 '2021년 3월 20일',
 '2021년 3월 27일',
 '2021년 4월 3일',
 '2021년 4월 10일',
 '2021년 4월 17일',
 '2021년 4월 24일',
 '2021년 5월 1일',
 '2021년 5월 8일',
 '2021년 5월 15일',
 '2021년 5월 22일',
 '2021년 5월 29일',
 '2021년 6월 5일',
 '2021년 6월 12일',
 '2021년 6월 19일',
 '2021년 6월 26일']

### 페이지 내 예약 iframe 부분으로 드라이버 대상을 바꿔줘야 드라이버가 아이템을 찾습니다.

In [144]:
driver.switch_to.frame(driver.find_element(By.XPATH, "//iframe[@src='https://eng.snu.ac.kr/room_reserve_page/index.php?uid=17907&name=7J2066+86riw&login_email=pz324250@snu.ac.kr&dept=6rK97JiB64yA7ZWZIOqyveyYge2VmeqzvA==']"))

### 아래가 예약하는 코드입니다.

In [149]:

while(True):    #끝날때까지 무한반복
    driver.find_element_by_class_name("fc-icon.fc-icon-right-single-arrow").click() # 다음날로 넘어가기
    time.sleep(1) 
    if(driver.find_element(By.XPATH, "//div[@class='fc-center']/h2").text in days): # 날짜 부분이 우리 세션하는 날짜면(평일 = 화목)

        # 해당하는 시간만큼 드래그하는 코드입니다
        ac = ActionChains(driver)
        # 아래 부분에서 move_offset다음에 숫자 두개가 각각 x좌표 y좌표인데, y좌표는 건드릴 필요 없고 x좌표로 돌아다니면서 원하는 시간이 숫자 몇에 위치해있는지 찾아야 합니다! 
        # 시간들이 따로 item으로 분리돼있지 않아서 좌표값을 설정하도록 해뒀어요
        # 호실은 data-resource-id=에서 바꾸시면 됩니다. 저는 404호라 '404'로 해뒀고 201호는 '201'뭐 이런식으로 채우시면 됩니다! id이름은 찾아보시면 될 것 같아요 ㅎㅎ
        ac.move_to_element(driver.find_element(By.XPATH, "//tr[@data-resource-id='404']")).move_by_offset(720, 0).click_and_hold().perform()  
        time.sleep(1)
        ac.move_by_offset(10, 0).release().perform()

        # 여기서부터는 알림창(예약하시겠습니까? - 예)을 해결하는 부분입니다. 알림창 예 누르고 나서 조금 쉬게 해뒀으니 그 사이에 제대로 예약됐나 확인하시면 됩니다!
        alert = driver.switch_to.alert
        time.sleep(1)
        alert.accept()
        time.sleep(3)
        print(driver.find_element(By.XPATH, "//div[@class='fc-center']/h2").text+" 완료")

        #토요일은 위랑 똑같긴 한데 예약 시간이 다르니 좌표만 다르게 설정해두시면 됩니다 :)
    if(driver.find_element(By.XPATH, "//div[@class='fc-center']/h2").text in sats):
        ac = ActionChains(driver)
        ac.move_to_element(driver.find_element(By.XPATH, "//tr[@data-resource-id='404']")).move_by_offset(423, 0).click_and_hold().perform()
        time.sleep(1)
        ac.move_by_offset(77, 0).release().perform()
        alert = driver.switch_to.alert
        time.sleep(1)
        alert.accept()
        time.sleep(3)
        print(driver.find_element(By.XPATH, "//div[@class='fc-center']/h2").text+" 완료")
    if(driver.find_element(By.XPATH, "//div[@class='fc-center']/h2").text =='2021년 7월 1일'):
        break        

2021년 4월 1일 완료
2021년 4월 3일 완료
2021년 4월 6일 완료
2021년 4월 8일 완료
2021년 4월 10일 완료
2021년 4월 13일 완료
2021년 4월 15일 완료
2021년 4월 17일 완료
2021년 4월 20일 완료
2021년 4월 22일 완료
2021년 4월 24일 완료
2021년 4월 27일 완료
2021년 4월 29일 완료
2021년 5월 1일 완료
2021년 5월 4일 완료
2021년 5월 6일 완료
2021년 5월 8일 완료
2021년 5월 11일 완료
2021년 5월 13일 완료
2021년 5월 15일 완료
2021년 5월 18일 완료
2021년 5월 20일 완료
2021년 5월 22일 완료
2021년 5월 25일 완료
2021년 5월 27일 완료
2021년 5월 29일 완료
2021년 6월 1일 완료
2021년 6월 3일 완료
2021년 6월 5일 완료
2021년 6월 8일 완료
2021년 6월 10일 완료
2021년 6월 12일 완료
2021년 6월 15일 완료
2021년 6월 17일 완료
2021년 6월 19일 완료
2021년 6월 22일 완료
2021년 6월 24일 완료
2021년 6월 26일 완료
2021년 6월 29일 완료


### 아래 하나씩만 클릭하는 코드입니다. 원하는 시간 좌표 찾을 때 사용하세용

In [ ]:
ac = ActionChains(driver)
ac.move_to_element(driver.find_element(By.XPATH, "//tr[@data-resource-id='404']")).move_by_offset(720, 0).click().perform() #

### 아래 코드는 예약 파트만 따로 빼둔 겁니다. 오류나서 하나하나 해야할 때 사용하세용

In [148]:
# 화목
ac = ActionChains(driver)
ac.move_to_element(driver.find_element(By.XPATH, "//tr[@data-resource-id='404']")).move_by_offset(720, 0).click_and_hold().perform()
time.sleep(1)
ac.move_by_offset(10, 0).release().perform()
time.sleep(1)
alert = driver.switch_to.alert
alert.accept()
print(driver.find_element(By.XPATH, "//div[@class='fc-center']/h2").text+" 완료")

2021년 3월 30일 완료


In [140]:
# 토요일
ac = ActionChains(driver)
ac.move_to_element(driver.find_element(By.XPATH, "//tr[@data-resource-id='404']")).move_by_offset(423, 0).click_and_hold().perform()
time.sleep(1)
ac.move_by_offset(77, 0).release().perform()\
alert = driver.switch_to.alert
alert.accept()
print(driver.find_element(By.XPATH, "//div[@class='fc-center']/h2").text+" 완료")

alert accepted


### 이동할 때 사용하세용 "fc-icon.fc-icon-left-single-arrow" 이부분에서 left면 앞으로가기 right면 뒤로가기

In [151]:
while(True):
    driver.find_element_by_class_name("fc-icon.fc-icon-left-single-arrow").click()
    if(driver.find_element(By.XPATH, "//div[@class='fc-center']/h2").text == "2021년 3월 13일"):
        break